In [118]:
import pandas as pd
import numpy as np

In [2]:
full_df = pd.read_csv('/Users/AngelaTam/Downloads/inputed50_cleaned.csv')
ref_df = pd.read_csv('/Users/AngelaTam/Downloads/inputed50_ref.csv')
d1d2 = pd.read_csv('/Users/AngelaTam/Downloads/tadpole_challenge/TADPOLE_D1_D2.csv')

In [13]:
full_df.drop(full_df.columns[0],axis=1,inplace=True)

In [79]:
full_df['RID'] = ref_df['RID']
full_df['VISCODE'] = ref_df['VISCODE']
full_df['COLPROT'] = ref_df['COLPROT']
full_df['EXAMDATE'] = ref_df['EXAMDATE']
full_df['Month_bl'] = d1d2['Month_bl']

In [80]:
data = full_df

target = ref_df[['y_DX','y_ADAS13','y_Ventricles_adj_TIV']]
#target = target.dropna(axis=0)
#data = data.loc[target.index]
#data = data.dropna(axis=1)
#target = target.loc[data.index]
target['Month_bl'] = data['Month_bl']
target = target[['Month_bl','y_DX','y_ADAS13','y_Ventricles_adj_TIV']]

/Users/AngelaTam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


"\ndata['VISCODE'][data['VISCODE']=='bl'] = 0\ndata['VISCODE'][data['VISCODE']=='m06'] = 0.6\ndata['VISCODE'][data['VISCODE']=='m12'] = 1\ndata['VISCODE'][data['VISCODE']=='m18'] = 1.6\ndata['VISCODE'][data['VISCODE']=='m24'] = 2\ndata['VISCODE'][data['VISCODE']=='m36'] = 3\ndata['VISCODE'][data['VISCODE']=='m48'] = 4\ndata['VISCODE'][data['VISCODE']=='m60'] = 5\ndata['VISCODE'][data['VISCODE']=='m72'] = 6\ndata['VISCODE'][data['VISCODE']=='m84'] = 7\ndata['VISCODE'][data['VISCODE']=='m96'] = 8\ndata['VISCODE'][data['VISCODE']=='m108'] = 9\ndata['VISCODE'][data['VISCODE']=='m120'] = 10\n"

In [99]:
for col in target.columns[1:]:
    data[col] = target[col]

In [124]:

def compute_data_table(sel_index):
    x_ = []
    y_ = []
    selection = data.iloc[sel_index][['RID','Month_bl']]
    for i in range(selection.shape[0]):
        rid, vis_month = selection.iloc[i].values
        #print(rid,vis_month)
        mask = (data[data['RID']==rid]['Month_bl']>vis_month).values

        valid_data = data[data['RID']==rid][mask]
        target_ = target[data['RID']==rid][mask]
        
        #print(mask)
        if valid_data.shape[0]>0:
            # add to x 
            x_.append(data.iloc[sel_index[i]].drop(['VISCODE','EXAMDATE','COLPROT','Month_bl']).values)
            # add to y
            target_ = target_.values
            target_[:,0] = target_[:,0]-vis_month
            #print(target_)
            
            y_.append(target_)
            
    return x_,y_
 


In [125]:
sel_index = data.index
x_,y_ = compute_data_table(sel_index) 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

